# Unit 5 - Financial Planning

In [1]:
# Initial imports
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
from alpaca_trade_api.rest import REST

%matplotlib inline


In [2]:
# Load .env enviroment variables
load_dotenv('example.env')


True

## Part 1 - Personal Finance Planner

### Collect Crypto Prices Using the `requests` Library

In [3]:
# Set current amount of crypto assets
my_btc = 1.2
my_eth = 5.3

In [4]:
# Crypto API URLs
btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=CAD"
eth_url = "https://api.alternative.me/v2/ticker/Ethereum/?convert=CAD"
btc_url

'https://api.alternative.me/v2/ticker/Bitcoin/?convert=CAD'

In [5]:
btc_url = btc_url + "?format=json"
eth_url = eth_url + "?format=json"

In [6]:
# Fetch current BTC price
btc_price = requests.get(btc_url).json()
btc_price = btc_price["data"]["1"]["quotes"]["USD"]["price"]

# Fetch current ETH price
eth_price = requests.get(eth_url).json()
eth_price = eth_price["data"]["1027"]["quotes"]["USD"]["price"]



# Compute current value of my crpto
my_btc_value = (my_btc * btc_price)
my_eth_value = (my_eth * eth_price)


# Print current crypto wallet balance
print(f"The current value of your {my_btc} BTC is ${my_btc_value:0.2f}")
print(f"The current value of your {my_eth} ETH is ${my_eth_value:0.2f}")

The current value of your 1.2 BTC is $52389.60
The current value of your 5.3 ETH is $17333.70


### Collect Investments Data Using Alpaca: `SPY` (stocks) and `AGG` (bonds)

In [7]:
# Set current amount of shares
my_agg = 200
my_spy = 50

In [8]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")


In [9]:
print(f"Alpaca Key type: {type(alpaca_api_key)}")
print(f"Alpaca Secret Key type: {type(alpaca_secret_key)}")

Alpaca Key type: <class 'str'>
Alpaca Secret Key type: <class 'str'>


In [10]:
# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2"
)

In [41]:
# Format current date as ISO format
start = pd.Timestamp("2015-09-10", tz="America/New_York").isoformat()
end = pd.Timestamp("2022-03-03", tz="America/New_York").isoformat()


# Set the tickers
tickers = ["AGG", "SPY"]

# Set timeframe to '1D' for Alpaca API
timeframe = "1Day"

# Get current closing prices for SPY and AGG
# (use a limit=1000 parameter to call the most recent 1000 days of data)
df_agg = alpaca.get_bars(
    tickers[0],
    timeframe,
    start = start,
   
    limit = 10000
).df                                    
  
df_spy = alpaca.get_bars(
    tickers[1],
    timeframe,
    start = start,
    
    limit = 10000
).df             

In [42]:
    
df_agg.head()
df_spy.head()

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2015-12-01 05:00:00+00:00,209.42,210.8200,209.1100,210.68,97858418,337780,209.921410
2015-12-02 05:00:00+00:00,210.60,211.0000,208.2300,208.54,108069059,367013,209.563055
2015-12-03 05:00:00+00:00,208.90,209.1500,204.7511,205.58,166224154,546768,206.878936
2015-12-04 05:00:00+00:00,206.10,209.9700,205.9300,209.66,192878747,556731,208.178631
2015-12-07 05:00:00+00:00,209.20,209.7295,207.2000,208.27,102027111,374705,208.276128


In [43]:
df_portfolio_year = df_agg.join(df_spy,lsuffix = "_AGG", rsuffix = "_SPY")

df_portfolio_year.tail()

,open_AGG,high_AGG,low_AGG,close_AGG,volume_AGG,trade_count_AGG,vwap_AGG,open_SPY,high_SPY,low_SPY,close_SPY,volume_SPY,trade_count_SPY,vwap_SPY
timestamp,,,,,,,,,,,,,,
2022-04-01 04:00:00+00:00,106.35,106.950,106.250,106.75,11579091,40126,106.667931,453.30,453.46,449.14,452.92,88676014,624745,451.850848
2022-04-04 04:00:00+00:00,106.87,106.870,106.560,106.84,9956493,32544,106.739298,453.20,456.91,452.26,456.80,59663209,503316,455.309774
2022-04-05 04:00:00+00:00,106.60,106.625,105.740,105.76,11018430,34684,105.931526,455.23,457.83,449.82,451.03,74022189,644313,453.412462
2022-04-06 04:00:00+00:00,105.20,105.805,105.115,105.49,13322630,47090,105.463015,446.87,448.93,443.47,446.52,105592595,901634,446.345270
2022-04-07 04:00:00+00:00,105.38,105.510,105.130,105.26,7921117,25631,105.278761,445.67,450.69,443.53,448.77,77538519,628673,447.098209


In [44]:
df_portfolio_year.columns


Index(['open_AGG', 'high_AGG', 'low_AGG', 'close_AGG', 'volume_AGG',
       'trade_count_AGG', 'vwap_AGG', 'open_SPY', 'high_SPY', 'low_SPY',
       'close_SPY', 'volume_SPY', 'trade_count_SPY', 'vwap_SPY'],
      dtype='object')

In [45]:
df_closing_prices = pd.DataFrame()

# Fetch the closing prices of AGG and SPY
df_closing_prices["AGG"] = df_portfolio_year["close_AGG"]
df_closing_prices["SPY"] = df_portfolio_year["close_SPY"]

# Drop the time component of the date
df_closing_prices.index = df_closing_prices.index.date

# Display sample data
df_closing_prices.tail(10)


,AGG,SPY
2022-03-25,106.10,452.69
2022-03-28,106.36,455.84
2022-03-29,106.86,461.53
2022-03-30,107.15,458.79
2022-03-31,107.10,452.11
2022-04-01,106.75,452.92
2022-04-04,106.84,456.80
2022-04-05,105.76,451.03
2022-04-06,105.49,446.52
2022-04-07,105.26,448.77


In [52]:
# Pick AGG and SPY close prices
agg_close_price = df_portfolio_year["close_AGG"].item
spy_close_price = df_portfolio_year["close_SPY"].item
print(agg_close_price)

<bound method IndexOpsMixin.item of timestamp
2015-12-01 05:00:00+00:00    108.82
2015-12-02 05:00:00+00:00    108.67
2015-12-03 05:00:00+00:00    107.89
2015-12-04 05:00:00+00:00    108.24
2015-12-07 05:00:00+00:00    108.40
                              ...  
2022-04-01 04:00:00+00:00    106.75
2022-04-04 04:00:00+00:00    106.84
2022-04-05 04:00:00+00:00    105.76
2022-04-06 04:00:00+00:00    105.49
2022-04-07 04:00:00+00:00    105.26
Name: close_AGG, Length: 1600, dtype: float64>


In [53]:
# Compute the current value of shares
my_spy_value = my_spy * spy_close_price
my_agg_value = my_agg * agg_close_price

# Print current value of shares
print(f"The current value of your {my_spy} SPY shares is ${my_spy_value:0.2f}")
print(f"The current value of your {my_agg} AGG shares is ${my_agg_value:0.2f}")

TypeError: unsupported operand type(s) for *: 'int' and 'method'

### Savings Health Analysis

In [55]:
# Set monthly household income
monthly_income = 12000

# Consolidate financial assets data
total_portfolio_dictionary = {'Amount': [total_btc_value+total_eth_value, my_spy_value+my_agg_value]}


# Create savings DataFrame
df_savings = pd.DataFrame(total_portfolio_dictionary,index=['Crypto','Shares'])


# Display savings DataFrame
display(df_savings)

NameError: name 'total_btc_value' is not defined

In [56]:
# Plot savings pie chart
df_savings.plot(kind='pie', subplots=True)

NameError: name 'df_savings' is not defined

In [57]:
# Set ideal emergency fund
emergency_fund = monthly_income * 3

# Calculate total amount of savings
total_savings = df_savings.sum().item()

# Validate saving health
if total_savings > emergency_fund:
    print("Great job on saving more than 3x your monthly income in your emergency fund.")
elif total_savings == emergency_fund:
    print("Great job on reaching your goal of 3x your monthly income in your emergency fund.")
else:
    print(f'You are ${emergency_fund - total_savings} away from reaching your goal of 3x your monthly income in your emergency fund.')

NameError: name 'df_savings' is not defined

## Part 2 - Retirement Planning

### Monte Carlo Simulation

In [58]:
# Set start and end dates of five years back from today.
# Sample results may vary from the solution based on the time frame chosen
start_date = pd.Timestamp('2016-05-01', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2021-05-01', tz='America/New_York').isoformat()

In [61]:
# Get 5 years' worth of historical data for SPY and AGG
df_stock_data = alpaca.get_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df


# Display sample data
df_stock_data.head()

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2016-05-02 04:00:00+00:00,110.74,110.7400,110.5200,110.61,3553939,15119,110.627199,AGG
2016-05-03 04:00:00+00:00,110.89,110.9450,110.7500,110.81,3044940,15662,110.865995,AGG
2016-05-04 04:00:00+00:00,110.91,110.9315,110.7595,110.86,2920608,13982,110.879063,AGG
2016-05-05 04:00:00+00:00,110.92,111.1600,110.8010,111.14,3008142,15707,111.028363,AGG
2016-05-06 04:00:00+00:00,111.12,111.1200,110.9700,110.97,2718710,12697,111.031967,AGG


In [73]:
# Configuring a Monte Carlo simulation to forecast 30 years cumulative returns
MC_thirty_year = MCSimulation(
    
    portfolio_data = df_stock_data,
    weights = [.40,.60],
    num_simulation = 500,
    num_trading_days = 252*30
)

IndexError: Too many levels: Index has only 1 level, not 2

In [66]:
# Printing the simulation input data
MC_thirty_year.portfolio_data.head()

NameError: name 'MC_thirty_year' is not defined

In [67]:
# Running a Monte Carlo simulation to forecast 30 years cumulative returns
MC_thirty_year.calc_cumulative_return()



NameError: name 'MC_thirty_year' is not defined

In [68]:
# Plot simulation outcomes
line_plot = MC_thirty_year.plot_simulation(

SyntaxError: unexpected EOF while parsing (3226889567.py, line 2)

In [69]:
# Plot probability distribution and confidence intervals
dist_plot = MC_thirty_year.plot_distribution()




NameError: name 'MC_thirty_year' is not defined

### Retirement Analysis

In [70]:
# Fetch summary statistics from the Monte Carlo simulation results
tbl = MC_thirty_year.summarize_cumulative_return()


# Print summary statistics
print(tbl)

NameError: name 'MC_thirty_year' is not defined

### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `$20,000` initial investment.

In [71]:
# Set initial investment
initial_investment = 20000

ci_lower = round(tbl[8]*initial_investment,2)
ci_upper = round(tbl[9]*initial_investment,2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

NameError: name 'tbl' is not defined

### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `50%` increase in the initial investment.

In [72]:
# Set initial investment
initial_investment = 20000 * 1.5

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $30,000
ci_lower = round(tbl[8]*initial_investment,2)
ci_upper = round(tbl[9]*initial_investment,2)


# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

NameError: name 'tbl' is not defined

## Optional Challenge - Early Retirement


### Five Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 5 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Running a Monte Carlo simulation to forecast 5 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Plot simulation outcomes
# YOUR CODE HERE!

In [ ]:
# Plot probability distribution and confidence intervals
# YOUR CODE HERE!

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
# YOUR CODE HERE!

# Print summary statistics
# YOUR CODE HERE!

In [ ]:
# Set initial investment
# YOUR CODE HERE!

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
# YOUR CODE HERE!

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${ci_lower_five} and ${ci_upper_five}")

### Ten Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 10 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Running a Monte Carlo simulation to forecast 10 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Plot simulation outcomes
# YOUR CODE HERE!

In [ ]:
# Plot probability distribution and confidence intervals
# YOUR CODE HERE!

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
# YOUR CODE HERE!

# Print summary statistics
# YOUR CODE HERE!

In [ ]:
# Set initial investment
# YOUR CODE HERE!

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
# YOUR CODE HERE!

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 10 years will end within in the range of"
      f" ${ci_lower_ten} and ${ci_upper_ten}")